In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', None)

In [2]:
## reading exported csv

# to get csv - save 'BEAM Deploy Status and Run Data' as csv
# if there is not enough permissions - save a copy and then save as csv

local_path = 'beam-production/jupyter/local_files/latest_all_runs.csv'
data = pd.read_csv(f"../../../{local_path}", parse_dates=['Time'])

# using only runs from specific data 
min_time = pd.to_datetime("2022-12-01") # yyyy-mm-dd
data = data[data['Time'] > min_time].copy()

print(f"there are roughly {len(data) / 2} runs since {min_time.strftime('%Y-%m-%d')}")
print(f"the latest run is from {data['Time'].max()}")

data['Month Period'] = data['Time'].dt.strftime('%Y-%m')
print(f"following data periods are included: {sorted(data['Month Period'].unique())}")

data.head(3)

there are roughly 371.0 runs since 2022-12-01
the latest run is from 2022-12-21 16:52:40
following data periods are included: ['2022-12']


,Status,Run Name,S3 Url,Instance type,Time,Host name,Web browser,Region,Batch,Branch,Commit,Data Branch,Data Commit,Instance ID,Config,Max RAM,Stacktrace,Died Actor,Error,Warning,SigOpt Client Id,SigOpt Dev Id,Profiler,Unnamed: 23,Month Period
16996,Run Started,sfbay-freight-calibration-4-batch8_10712736+htmlrbATusers_noreply_github_com,NaN,r5d.24xlarge,2022-12-01 07:44:13,ec2-18-224-199-181.us-east-2.compute.amazonaws.com,http://ec2-18-224-199-181.us-east-2.compute.amazonaws.com:8000,us-east-2,1ded3732,freight-develop-merge,fe1904ad0290c746ecaf39b91b1a1fafa8d68986,develop,4517e0476102c7fc115ce2a46c0a1509362c96f3,i-0286b968d8686bca3,production/sfbay/freight/sfbay-freight-calibration-4.conf,740g,NaN,NaN,NaN,NaN,LVJBLBUUDVULXEAGEQVKUNHZEUSLBDYHEQBIMKEURFEKDWMA,EZLHTZTCVNWIKGTTTEWYXIILFYAMVGSUVIBVYPOHNAZPVEJK,cpumem,NaN,2022-12
16997,Run Started,sfbay-freight-calibration-3-batch8_10712736+htmlrbATusers_noreply_github_com,NaN,r5d.24xlarge,2022-12-01 07:44:13,ec2-18-219-191-182.us-east-2.compute.amazonaws.com,http://ec2-18-219-191-182.us-east-2.compute.amazonaws.com:8000,us-east-2,94efe454,freight-develop-merge,fe1904ad0290c746ecaf39b91b1a1fafa8d68986,develop,4517e0476102c7fc115ce2a46c0a1509362c96f3,i-0aac5e56939e0726c,production/sfbay/freight/sfbay-freight-calibration-3.conf,740g,NaN,NaN,NaN,NaN,LVJBLBUUDVULXEAGEQVKUNHZEUSLBDYHEQBIMKEURFEKDWMA,EZLHTZTCVNWIKGTTTEWYXIILFYAMVGSUVIBVYPOHNAZPVEJK,cpumem,NaN,2022-12
16998,Run Started,sfbay-freight-calibration-2-batch8_10712736+htmlrbATusers_noreply_github_com,NaN,r5d.24xlarge,2022-12-01 07:44:13,ec2-18-118-145-201.us-east-2.compute.amazonaws.com,http://ec2-18-118-145-201.us-east-2.compute.amazonaws.com:8000,us-east-2,d93338e7,freight-develop-merge,fe1904ad0290c746ecaf39b91b1a1fafa8d68986,develop,4517e0476102c7fc115ce2a46c0a1509362c96f3,i-0b0c8ecc89f7933fc,production/sfbay/freight/sfbay-freight-calibration-2.conf,740g,NaN,NaN,NaN,NaN,LVJBLBUUDVULXEAGEQVKUNHZEUSLBDYHEQBIMKEURFEKDWMA,EZLHTZTCVNWIKGTTTEWYXIILFYAMVGSUVIBVYPOHNAZPVEJK,cpumem,NaN,2022-12


In [3]:
## getting data frame with each row as one simulation

take_first_columns = ['Run Name','Month Period','Branch','Instance type']

df = data.groupby("Host name").agg(list)
for col in take_first_columns:
    df[col] = df.apply(lambda r: r[col][0], axis=1)

df['Time Start'] = df.apply(lambda r: r['Time'][0], axis=1)
df['Time Stop'] = df.apply(lambda r: r['Time'][-1], axis=1)
df['Status'] = df.apply(lambda r: r['Status'][-1], axis=1)

all_columns = set(df.columns)
taken_columns = take_first_columns + ['Time Start', 'Time Stop', 'Status']

df = df[taken_columns].copy()

removed_columns = list(sorted(all_columns - set(taken_columns)))
half_len = int(len(removed_columns)/2)
print(f"removed columns: {removed_columns}")

# fix for some wierd shift in the spreadsheet for few rows
for v in ['ec2-18-221-208-40.us-east-2.compute.amazonaws.com','ec2-3-144-69-95.us-east-2.compute.amazonaws.com','ec2-52-15-53-101.us-east-2.compute.amazonaws.com']:
    df.replace(to_replace=v, value='r5d.24xlarge', inplace=True)

df['duration_hours'] = (df['Time Stop'] - df['Time Start']).astype('timedelta64[h]')

df.head(3)

removed columns: ['Batch', 'Commit', 'Config', 'Data Branch', 'Data Commit', 'Died Actor', 'Error', 'Instance ID', 'Max RAM', 'Profiler', 'Region', 'S3 Url', 'SigOpt Client Id', 'SigOpt Dev Id', 'Stacktrace', 'Time', 'Unnamed: 23', 'Warning', 'Web browser']


,Run Name,Month Period,Branch,Instance type,Time Start,Time Stop,Status,duration_hours
Host name,,,,,,,,
ec2-13-58-102-130.us-east-2.compute.amazonaws.com,sfbay-freight-calibration-5-batch8_10712736+htmlrbATusers_noreply_github_com,2022-12,freight-develop-merge,r5d.24xlarge,2022-12-01 07:44:14,2022-12-01 23:26:03,Run Failed,15.0
ec2-13-58-111-254.us-east-2.compute.amazonaws.com,sfbay-micro-mobility-0.3pop-cali25_j503440616ATberkeley_edu,2022-12,zn/tour-mode-with-new-stall-sampling,r5d.24xlarge,2022-12-12 17:30:25,2022-12-13 03:25:43,Run Completed,9.0
ec2-13-58-202-129.us-east-2.compute.amazonaws.com,new-york-may2020-part-0-updateGTFS-updateCapacities9_cpolizianiATlbl_gov,2022-12,inm/nyc-code-change-with-plans-generation,r5d.16xlarge,2022-12-20 05:13:36,2022-12-20 05:13:36,Run Completed,0.0


In [4]:
## calculating a price in USD of each simulation

instance_to_price = {
    'c5d.24xlarge' : 4.608,
    'c6a.24xlarge' : 3.672,
    'hpc6a.48xlarge' : 2.88,
    'm4.16xlarge' : 3.2,
    'm5.12xlarge' : 2.304,
    'm5.24xlarge' : 4.608,
    'm5d.24xlarge' : 5.424,
    'r5.24xlarge' : 6.048,
    'r5.2xlarge' : 0.504,
    'r5.4xlarge' : 1.008,
    'r5.8xlarge' : 2.016,
    'r5.large' : 0.126,
    'r5.xlarge' : 0.252,
    'r5d.12xlarge' : 3.456,
    'r5d.16xlarge' : 4.608,
    'r5d.24xlarge' : 6.912,
    't2.medium' : 0.0464                 
}

# for instance_type in sorted(instance_to_price.keys()):
#     print(f"'{instance_type}' : {instance_to_price[instance_type]},")

missing_instance_types = set()
def get_instance_hour_cost(row):
    instance_type = row['Instance type']
    if instance_type in instance_to_price :
        return instance_to_price[instance_type]

    missing_instance_types.add(instance_type)
    return 0.0

df['aws_instance_hour_cost'] = df.apply(get_instance_hour_cost, axis=1)

if len(missing_instance_types) > 0:
    print(f"Can't find price for {len(missing_instance_types)} instance types.")
    for missing_instance in missing_instance_types:
        print(f"'{missing_instance}': ,")
    
df['cost'] = df['duration_hours'] * df['aws_instance_hour_cost']
total_cost = int(df['cost'].sum())


budget_amount_used_from_aws = 35268.65
def print_total_info():
    dt_interval = f"from {min_time.strftime('%Y-%m-%d')} to {data['Time'].max().strftime('%Y-%m-%d')}"
    print(f"There are {len(df)} simulations {dt_interval}")
    delta = int(budget_amount_used_from_aws - total_cost)
    print(f"The total cost of all instances time is ${total_cost}, amount of calculated by AWS: ${budget_amount_used_from_aws} [${delta} unrecorded on our side]")

print_total_info()
    
df.head(3)

Can't find price for 1 instance types.
't2.small': ,
There are 408 simulations from 2022-12-01 to 2022-12-21
The total cost of all instances time is $30892, amount of calculated by AWS: $35268.65 [$4376 unrecorded on our side]


,Run Name,Month Period,Branch,Instance type,Time Start,Time Stop,Status,duration_hours,aws_instance_hour_cost,cost
Host name,,,,,,,,,,
ec2-13-58-102-130.us-east-2.compute.amazonaws.com,sfbay-freight-calibration-5-batch8_10712736+htmlrbATusers_noreply_github_com,2022-12,freight-develop-merge,r5d.24xlarge,2022-12-01 07:44:14,2022-12-01 23:26:03,Run Failed,15.0,6.912,103.680
ec2-13-58-111-254.us-east-2.compute.amazonaws.com,sfbay-micro-mobility-0.3pop-cali25_j503440616ATberkeley_edu,2022-12,zn/tour-mode-with-new-stall-sampling,r5d.24xlarge,2022-12-12 17:30:25,2022-12-13 03:25:43,Run Completed,9.0,6.912,62.208
ec2-13-58-202-129.us-east-2.compute.amazonaws.com,new-york-may2020-part-0-updateGTFS-updateCapacities9_cpolizianiATlbl_gov,2022-12,inm/nyc-code-change-with-plans-generation,r5d.16xlarge,2022-12-20 05:13:36,2022-12-20 05:13:36,Run Completed,0.0,4.608,0.000


In [5]:
## applying 'project' to the list of simulations based on simulation name and|or git branch name

def get_owner(row):
    run_name = row['Run Name']
    if '/' in run_name:
        return run_name.split('/')[0]
    return run_name


def get_branch_owner(row):
    branch = row['Branch'].split('/')
    if len(branch) > 1:
        return branch[0]
    return branch


def get_project(row):
    owner = get_owner(row)
    branch_owner = get_branch_owner(row)
    project = f"{owner} | {branch_owner}".lower()

    if 'new-york' in project:
        return f"NYC"
    if 'freight' in project:
        return "Freight"
    if 'gemini' in project:
        return "Gemini"
    if 'micro-mobility' in project or 'micromobility' in project:
        return "micro-mobility"
    if 'shared' in project:
        return "shared fleet"
    if 'profiling' in project:
        return "CPU profiling"
    
    return project


df["project"] = df.apply(get_project, axis=1)
df.head(2)

,Run Name,Month Period,Branch,Instance type,Time Start,Time Stop,Status,duration_hours,aws_instance_hour_cost,cost,project
Host name,,,,,,,,,,,
ec2-13-58-102-130.us-east-2.compute.amazonaws.com,sfbay-freight-calibration-5-batch8_10712736+htmlrbATusers_noreply_github_com,2022-12,freight-develop-merge,r5d.24xlarge,2022-12-01 07:44:14,2022-12-01 23:26:03,Run Failed,15.0,6.912,103.680,Freight
ec2-13-58-111-254.us-east-2.compute.amazonaws.com,sfbay-micro-mobility-0.3pop-cali25_j503440616ATberkeley_edu,2022-12,zn/tour-mode-with-new-stall-sampling,r5d.24xlarge,2022-12-12 17:30:25,2022-12-13 03:25:43,Run Completed,9.0,6.912,62.208,micro-mobility


In [6]:
### processing simulations in unknown state, i.e. with 'Run Started' status
def get_fixed_status(row):
    status = row["Status"]
    if status != 'Run Started':
        return status
    inactive_time = pd.Timestamp.now() - row['Time Start']
    if inactive_time.days > 2:
        return 'Run Failed'
    return 'Maybe Running'

df['Status Fixed'] = df.apply(get_fixed_status, axis=1)


### grouping dataframe by project
df_grouped = df.groupby("project").agg(list).reset_index()

df_grouped["Instance time cost"] = df_grouped.apply(lambda r: sum(r['cost']), axis=1)
df_grouped["Fraction of total cost"] = df_grouped.apply(lambda r: r['Instance time cost'] / total_cost, axis=1)
df_grouped = df_grouped.sort_values("Fraction of total cost", ascending=False).reset_index()


def failed_runs_time_cost(project_row):
    runs_state = project_row['Status Fixed']
    runs_cost = project_row['cost']
    failed_cost_sum = 0.0
    for (state, cost) in zip(runs_state, runs_cost):
        if state == 'Run Failed':
            failed_cost_sum += cost
    return failed_cost_sum

df_grouped["Failed runs time cost"] = df_grouped.apply(failed_runs_time_cost, axis=1)


df_grouped["Instance types"] = df_grouped.apply(lambda r: list(set(r["Instance type"])), axis=1)

# 'Run Failed', 'Run Completed', 'Run Started', 'Unable to start'
df_grouped["Failed runs"] = df_grouped.apply(lambda r: r['Status Fixed'].count('Run Failed')+r['Status'].count('Unable to start'), axis=1)
df_grouped["Completed runs"] = df_grouped.apply(lambda r: r['Status Fixed'].count('Run Completed'), axis=1)
df_grouped["Maybe still running"] = df_grouped.apply(lambda r: r['Status Fixed'].count('Maybe Running'), axis=1)


columns_with_numbers = ["Instance time cost", "Failed runs time cost", "Fraction of total cost", "Completed runs", "Failed runs", "Maybe still running"]
df_grouped.loc["Total"] = df_grouped[columns_with_numbers].sum()

selected_columns = ["project", "Instance types"] + columns_with_numbers

print_total_info()
df_grouped[selected_columns]

There are 408 simulations from 2022-12-01 to 2022-12-21
The total cost of all instances time is $30892, amount of calculated by AWS: $35268.65 [$4376 unrecorded on our side]


,project,Instance types,Instance time cost,Failed runs time cost,Fraction of total cost,Completed runs,Failed runs,Maybe still running
0,micro-mobility,[r5d.24xlarge],13810.176,0.000,0.447047,120.0,9.0,0.0
1,NYC,[r5d.16xlarge],8626.176,1612.800,0.279237,78.0,78.0,16.0
2,Freight,[r5d.24xlarge],6559.488,1783.296,0.212336,14.0,14.0,0.0
3,shared fleet,[r5d.24xlarge],1140.480,836.352,0.036918,13.0,5.0,0.0
4,CPU profiling,"[c6a.24xlarge, r5d.24xlarge, hpc6a.48xlarge, c5d.24xlarge, m5.12xlarge]",307.944,307.944,0.009968,0.0,7.0,0.0
5,Gemini,[r5d.24xlarge],248.832,228.096,0.008055,1.0,8.0,0.0
6,alex-vv | alex-vv,"[c6a.24xlarge, r5d.24xlarge, hpc6a.48xlarge, c5d.24xlarge, m5.12xlarge]",158.904,34.560,0.005144,6.0,4.0,0.0
7,edward | edward,"[r5.8xlarge, r5d.24xlarge, r5.4xlarge, r5.large]",33.696,0.000,0.001091,20.0,5.0,0.0
8,inm | hl,[r5d.12xlarge],6.912,6.912,0.000224,0.0,4.0,0.0
9,0.1pop-totalbike-cali13_j503440616atberkeley_edu | xuan,[r5d.24xlarge],0.000,0.000,0.000000,0.0,1.0,0.0


In [10]:
print_total_info()
df_grouped[["project","Fraction of total cost"]]

There are 408 simulations from 2022-12-01 to 2022-12-21
The total cost of all instances time is $30892, amount of calculated by AWS: $35268.65 [$4376 unrecorded on our side]


,project,Fraction of total cost
0,micro-mobility,0.447047
1,NYC,0.279237
2,Freight,0.212336
3,shared fleet,0.036918
4,CPU profiling,0.009968
5,Gemini,0.008055
6,alex-vv | alex-vv,0.005144
7,edward | edward,0.001091
8,inm | hl,0.000224
9,0.1pop-totalbike-cali13_j503440616atberkeley_edu | xuan,0.000000


In [7]:
df_grouped_by_instance = df.groupby('Instance type').agg(list).reset_index()
# ['Instance type', 'Run Name', 'Month Period', 'Branch', 'Time Start', 'Time Stop', 'Status', 'duration_hours', 'aws_instance_hour_cost', 'cost', 'project', 'Status Fixed']

df_grouped_by_instance['Total cost per instance'] = df_grouped_by_instance.apply(lambda r: sum(r['cost']), axis=1)
df_grouped_by_instance = df_grouped_by_instance.sort_values('Total cost per instance', ascending=False).reset_index()

print_total_info()
df_grouped_by_instance[['Instance type', 'Total cost per instance']]

There are 408 simulations from 2022-12-01 to 2022-12-21
The total cost of all instances time is $30892, amount of calculated by AWS: $35268.65 [$4376 unrecorded on our side]


,Instance type,Total cost per instance
0,r5d.24xlarge,22035.456
1,r5d.16xlarge,8626.176
2,c5d.24xlarge,115.200
3,c6a.24xlarge,44.064
4,m5.12xlarge,29.952
5,hpc6a.48xlarge,28.800
6,r5d.12xlarge,6.912
7,r5.8xlarge,6.048
8,r5.2xlarge,0.000
9,r5.4xlarge,0.000
